In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))

# --- Imports ---
import asyncio
import pandas as pd
from openai import OpenAI, AsyncOpenAI
from langchain_openai import ChatOpenAI
from langchain_neo4j import Neo4jGraph

from util.config_loader import load_config_api
from util.api_client import ApiClient
from llm.tool import (
    build_ontology_mapper_tool,
    build_patient_ner_tool,
    build_patient_ned_tool,
    build_general_medical_tool,
    build_patient_info_tool,
    build_patient_coverage_tool,
)


In [3]:
from IPython.display import JSON, Markdown, display
import json

def show_json(raw, mode="text", indent=2):
    """
    mode = "tree"  -> interactive JSON widget
    mode = "text"  -> indented JSON
    mode = "both"  -> tree + indented JSON
    """
    if isinstance(raw, str):
        raw = json.loads(raw)

    if mode in ("tree", "both"):
        display(JSON(raw))

    if mode in ("text", "both"):
        pretty = json.dumps(raw, indent=indent, ensure_ascii=False)
        display(Markdown(f"```json\n{pretty}\n```"))

# Connectors

In [4]:
url_emb = load_config_api("embedding", path="../config.ini")
api = ApiClient(url_emb)
body, status, headers = api.post('/embed', {'input': ['Who are you?', 'What is your name?']})
print("Status:", status)
print("Headers:", headers)
print("Body:", body)

Status: 200
Headers: {'Content-Length': '153883', 'Content-Type': 'application/json', 'Date': 'Wed, 19 Nov 2025 13:26:29 GMT', 'Server': 'uvicorn', 'Connection': 'close'}
Body: {'model': 'Alibaba-NLP/gte-Qwen2-7B-instruct', 'num_inputs': 2, 'embedding_dims': 3584, 'data': [[0.006518279667943716, 0.0009311828180216253, -0.00012745565618388355, 0.01802769862115383, 0.003128774231299758, -0.009758795611560345, 0.013558021746575832, 0.0015736989444121718, -0.021901419386267662, 0.02413625828921795, 0.007561204489320517, -0.014973419718444347, 0.004916645120829344, -0.006406537722796202, 0.023242322728037834, 0.010503741912543774, 0.02160344086587429, -0.019666580483317375, -0.007002494763582945, -0.00506563438102603, 0.019666580483317375, 0.0021975913550704718, -0.0076356991194188595, -0.00030496236286126077, 0.006294795777648687, -0.016090840101242065, -0.015420387499034405, -0.0324796587228775, 0.004432430025190115, 0.02309333346784115, 0.01720825769007206, 0.004283440764993429, 0.002057

In [5]:
url_llm = load_config_api("llm", path="../config.ini")
chat_client = ChatOpenAI(
    api_key="EMPTY",
    base_url=url_llm,
    model_name="google/medgemma-4b-it",
    temperature=0,
    max_tokens=8192,
    top_p=0.9,
    frequency_penalty=0.2,
    presence_penalty=0.0,
)

In [6]:
graph_client = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="password",
    database="cdl2025",
    enhanced_schema=True,
)

# Open LLM Tests and Virtualized Resources


## Single Request

In [7]:
client = OpenAI(
        api_key="EMPTY",                      
        base_url=url_llm
    )
resp = client.chat.completions.create(
    model="google/medgemma-4b-it",
    messages=[
        {"role": "user", "content": "Who are you?"}
    ],
    temperature=0,
)
response = resp.choices[0].message.content
response

'I am Gemma, an open-weights AI assistant. I am a large language model trained by Google DeepMind.\n'

## Batch Request


In [8]:
prompts = [
    "Explain the difference between type 1 and type 2 diabetes in simple terms. In one sentence.",
    "List the main symptoms of pneumonia and when a patient should see a doctor. In one sentence.",
    "What lifestyle changes can help reduce high blood pressure? In one sentence.",
    "Describe the importance of vaccination in preventing infectious diseases. In one sentence.",
    "What are the common treatments for seasonal allergies? In one sentence.",
    "How does regular exercise benefit mental health? In one sentence.",
    "What dietary changes can help manage cholesterol levels? In one sentence.",
]

async def run_batch(prompts, concurrency=8, max_tokens=128):
    client = AsyncOpenAI(api_key="EMPTY", base_url=url_llm)
    sem = asyncio.Semaphore(concurrency)

    async def one(p):
        async with sem:
            r = await client.chat.completions.create(
                model="google/medgemma-4b-it",
                messages=[{"role": "user", "content": p}],
                temperature=0.0,
                max_tokens=max_tokens,
                stream=False,
            )
            return r.choices[0].message.content

    tasks = [asyncio.create_task(one(p)) for p in prompts]
    out = await asyncio.gather(*tasks)
    await client.close()
    return out

answers = await run_batch(prompts, concurrency=8)
for q, a in zip(prompts, answers):
    print(q, "→", a)


Explain the difference between type 1 and type 2 diabetes in simple terms. In one sentence. → Type 1 diabetes is an autoimmune disease where the body attacks and destroys insulin-producing cells, while type 2 diabetes is a condition where the body doesn't use insulin properly, often due to lifestyle factors.

List the main symptoms of pneumonia and when a patient should see a doctor. In one sentence. → Pneumonia symptoms include cough (often with phlegm), fever, chills, shortness of breath, and chest pain. See a doctor if you have these symptoms, especially if you are elderly, have a chronic condition, or are immunocompromised.

What lifestyle changes can help reduce high blood pressure? In one sentence. → Lifestyle changes like diet modifications (reducing sodium and increasing potassium), regular exercise, maintaining a healthy weight, limiting alcohol consumption, and quitting smoking can significantly reduce high blood pressure.

Describe the importance of vaccination in preventing

## Virtualized Resources

### Materialize Data

In [9]:
graph_client.query("MATCH (n:HpoPhenotype) RETURN n.label OFFSET 100 LIMIT 10 ")

[{'n.label': 'Thick lower lip vermilion'},
 {'n.label': 'Abnormal uvula morphology'},
 {'n.label': 'Microglossia'},
 {'n.label': 'Abnormal palate morphology'},
 {'n.label': 'Broad alveolar ridges'},
 {'n.label': 'Narrow palate'},
 {'n.label': 'Short upper lip'},
 {'n.label': 'Lobulated tongue'},
 {'n.label': 'Tongue muscle weakness'},
 {'n.label': 'Movement abnormality of the tongue'}]

### Virtualized Data

In [10]:
pd.read_csv("../data/patient03.csv").head(5)

,PatientID,EncounterID,Encounter.period.start,Encounter.class,Encounter.reasonCode,ChiefComplaint,Condition,Comorbidities,Observation[vitals],Observation[key],DiagnosticReport,Procedure,MedicationStatement,Encounter.diagnosis.rank,Encounter.hospitalization.dischargeDisposition,Plan/FollowUp,CourseTrend,Notes,Narrative
0,P003,E001,2017-06-02,Ambulatory,Progressive leg tingling and burning,Leg numbness and electric-like pains,Multiple mononeuropathy,NaN,"137/81 mmHg, HR 78 bpm, RR 16, Temp 37.1°C, Sp...",Neurologic exam: patchy distal sensory loss an...,NCS/EMG: multifocal demyelinating features in ...,Nerve conduction study and EMG,Gabapentin 100 mg PO BID,1,Home,Neurology follow-up in 4 weeks,Worsened,Progressive sensory symptoms,The patient reported several weeks of burning ...
1,P003,E002,2018-03-15,Emergency,Severe headache with nausea,Persistent pressure-like headache and nausea,"Aseptic meningitis, fungal etiology suspected",Multiple mononeuropathy,"139/85 mmHg, HR 92 bpm, RR 18, Temp 38.0°C, Sp...","Neuro exam: mild neck stiffness, no focal defi...","CSF analysis: clear fluid, lymphocytic pleocyt...",Lumbar puncture and CT head,Ondansetron 4 mg IV PRN,1,Observation,Monitor neurologic status and repeat CSF if sy...,Improved,Headache and nausea partially relieved,The patient presented to the emergency departm...
2,P003,E003,2019-01-29,Ambulatory,Jaw pain and stress-related behaviors,Teeth grinding and morning jaw soreness,Bruxism,"Multiple mononeuropathy, prior aseptic meningitis","132/80 mmHg, HR 76 bpm, RR 16, Temp 36.8°C, Sp...",Oral exam: dental wear patterns and tender mas...,Behavioral assessment: stress-linked bruxism d...,Behavioral and dental evaluation,Hydroxyzine 25 mg PO PRN for anxiety,3,Home,Dental guard fitting and referral to behaviora...,Stable,Stress-related nocturnal grinding,The patient reported several months of waking ...
3,P003,E004,2019-10-12,Ambulatory,Eye pain and blurred vision,"Blurred vision, floaters, and eye discomfort",Posterior uveitis,"Multiple mononeuropathy, bruxism, prior asepti...","128/79 mmHg, HR 74 bpm, RR 17, Temp 36.7°C, Sp...",Ophthalmic exam: vitreous cells and mild retin...,Ophthalmology report: unilateral posterior uve...,Dilated ophthalmologic examination,Prednisolone acetate 1% ophthalmic drops QID t...,2,Home,Follow-up with ophthalmology in 1 week,Improved,Posterior segment inflammation partially respo...,The patient presented with several days of blu...
4,P003,E005,2020-08-03,Emergency,Palpitations and dizziness,Sudden pounding heartbeat with lightheadedness,Paroxysmal tachycardia,"Multiple mononeuropathy, posterior uveitis","142/88 mmHg, HR 148 bpm, RR 19, Temp 36.9°C, S...","ECG: narrow-complex tachycardia, spontaneously...",ED ECG report: short run of paroxysmal suprave...,12-lead ECG and cardiac monitoring,Metoprolol 25 mg PO daily,1,Home,Cardiology follow-up within 2 weeks,Improved,Rhythm stabilized after spontaneous conversion,"While walking up a short flight of stairs, the..."


In [11]:
graph_client.query("MATCH (n:Patient) RETURN n.label")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Patient)} {position: line: 1, column: 10, offset: 9} for query: 'MATCH (n:Patient) RETURN n.label'


[]

In [ ]:
graph_client.query("""
    CALL apoc.dv.query('patient', {patientId:'P003'}) YIELD node AS v
    RETURN v LIMIT 1
""")

[{'v': {'Condition': 'Multiple mononeuropathy',
   'Encounter.period.start': '2017-06-02',
   'EncounterID': 'E001',
   'Encounter.reasonCode': 'Progressive leg tingling and burning',
   'Encounter.hospitalization.dischargeDisposition': 'Home',
   'Encounter.class': 'Ambulatory',
   'Observation[key]': 'Neurologic exam: patchy distal sensory loss and reduced vibration sense in both feet',
   'DiagnosticReport': 'NCS/EMG: multifocal demyelinating features in lower limb nerves',
   'NED_Entities': '[{\'source\': \'concat\', \'start\': 14, \'end\': 24, \'text\': \'leg tingling\', \'label\': \'Diseases of the nervous system\', \'assertion\': \'present\', \'temporality\': \'acute\', \'rationale\': \'Patient reports leg tingling.\', \'icd_id\': \'G57.1\', \'icd_label\': \'Meralgia paraesthetica\', \'confidence\': 0.95, \'linking_rationale\': "The mention of \'leg tingling\' aligns with the ICD-10 code G57.1 (Meralgia paraesthetica), which describes the tingling sensation caused by compressio

# Ontology in the Knowledge Graph

## HPO Example

In [13]:
graph_client.query("""
    MATCH (p:HpoPhenotype {label: "Abnormality of the endocrine system"})<-[:HAS_PHENOTYPIC_FEATURE]-(d:HpoDisease)
    RETURN  d.label AS disease
""")

[{'disease': 'Biemond syndrome II'},
 {'disease': 'Pseudovaginal perineoscrotal hypospadias'},
 {'disease': 'Ectodermal dysplasia with adrenal cyst'},
 {'disease': 'Thymic-Renal-Anal-Lung dysplasia'},
 {'disease': 'Myasthenia gravis'},
 {'disease': 'Candidiasis, familial chronic mucocutaneous, autosomal dominant'},
 {'disease': 'PYGMY'},
 {'disease': 'PURA-related severe neonatal hypotonia-seizures-encephalopathy syndrome'},
 {'disease': 'Semilobar holoprosencephaly'},
 {'disease': 'Multiple endocrine neoplasia type 4'},
 {'disease': 'Fibrous dysplasia of bone'},
 {'disease': 'African trypanosomiasis'},
 {'disease': 'Midline interhemispheric variant of holoprosencephaly'},
 {'disease': 'Lobar holoprosencephaly'},
 {'disease': 'Alobar holoprosencephaly'},
 {'disease': 'Bardet-Biedl syndrome'},
 {'disease': 'Coccidioidomycosis'},
 {'disease': 'Isolated osteopoikilosis'},
 {'disease': 'Acromegaly'},
 {'disease': 'Immune dysregulation-polyendocrinopathy-enteropathy-X-linked syndrome'},
 {'

### Neosemantics for RDFS rdfs:subClassOf Inference

In [14]:
graph_client.query("""
MATCH (cat:HpoPhenotype {label: "Abnormality of the endocrine system"})
CALL n10s.inference.nodesInCategory(cat, { 
    inCatRel: "HAS_PHENOTYPIC_FEATURE", subCatRel: "SUBCLASSOF"})
YIELD node as dis
MATCH (dis)-[:HAS_PHENOTYPIC_FEATURE]->(phe:HpoPhenotype)
RETURN dis.label as disease, collect(DISTINCT phe.label) as features
ORDER BY size(features) ASC, disease
SKIP 100
LIMIT 5
""")

[{'disease': 'Edema, familial idiopathic, prepubertal',
  'features': ['Edema',
   'Abnormality of the genitourinary system',
   'Autosomal dominant inheritance',
   'Diabetes mellitus',
   'Irritability',
   'Vomiting']},
 {'disease': 'Familial dysalbuminemic hyperthyroxinemia',
  'features': ['Autosomal recessive inheritance',
   'Autosomal dominant inheritance',
   'Abnormal thyroid-stimulating hormone level',
   'Euthyroid hyperthyroxinemia',
   'Increased circulating free T4 concentration',
   'Abnormal circulating free T4 concentration']},
 {'disease': 'Hyperaldosteronism, familial, type II',
  'features': ['Autosomal dominant inheritance',
   'Increased circulating aldosterone concentration',
   'Hypertension',
   'Juvenile onset',
   'Hypokalemia',
   'Typified by incomplete penetrance']},
 {'disease': 'Hyperinsulinemic hypoglycemia, familial, 7',
  'features': ['Autosomal dominant inheritance',
   'Hyperinsulinemia',
   'Hyperinsulinemic hypoglycemia',
   'Hypoglycemia',
   'H

# Ontology Mapping

## Ontology Mapping Based on UMLS

In [15]:
graph_client.query("""
MATCH path = (d:IcdDisease)<-[:UMLS_TO_ICD]-(:UMLS)-[:UMLS_TO_HPO_PHENOTYPE]->(p:HpoPhenotype)
WITH path, d, p,
     relationships(path) AS path_edges,
     nodes(path)         AS path_nodes
WITH
  // always get some string for each node
  [n IN path_nodes |
     COALESCE(n.label, n.id, elementId(n))
  ] AS node_names,
  [r IN path_edges | type(r)] AS rel_types,
  [r IN path_edges |
     COALESCE(
       startNode(r).label,
       startNode(r).id,
       elementId(startNode(r))
     )
  ] AS rel_starts,
  d, p
WITH [i IN range(0, size(node_names) - 1) |
  CASE
    WHEN i = size(node_names) - 1
      THEN '(' + node_names[i] + ')'
    WHEN node_names[i] = rel_starts[i]
      THEN '(' + node_names[i] + ')' + '-[:' + rel_types[i] + ']->'
    ELSE '(' + node_names[i] + ')' + '<-[:' + rel_types[i] + ']-'
  END
] AS string_paths, d, p
RETURN DISTINCT
  apoc.text.join(string_paths, '') AS `Extracted path`
LIMIT 10;
""")

[{'Extracted path': '(Paratyphoid fever, unspecified)<-[:UMLS_TO_ICD]-(C0015672)-[:UMLS_TO_HPO_PHENOTYPE]->(Fatigue)'},
 {'Extracted path': '(Other salmonella infections)<-[:UMLS_TO_ICD]-(C0085593)-[:UMLS_TO_HPO_PHENOTYPE]->(Chills)'},
 {'Extracted path': '(Shigellosis)<-[:UMLS_TO_ICD]-(C0015967)-[:UMLS_TO_HPO_PHENOTYPE]->(Fever)'},
 {'Extracted path': '(Enterocolitis due to Clostridium difficile)<-[:UMLS_TO_ICD]-(C0238106)-[:UMLS_TO_HPO_PHENOTYPE]->(Clostridium difficile colitis)'},
 {'Extracted path': '(Other bacterial foodborne intoxications, not elsewhere classified)<-[:UMLS_TO_ICD]-(C0231218)-[:UMLS_TO_HPO_PHENOTYPE]->(Malaise)'},
 {'Extracted path': '(Amoebiasis)<-[:UMLS_TO_ICD]-(C0039070)-[:UMLS_TO_HPO_PHENOTYPE]->(Syncope)'},
 {'Extracted path': '(Amoebiasis)<-[:UMLS_TO_ICD]-(C0039070)-[:UMLS_TO_HPO_PHENOTYPE]->(Loss of consciousness)'},
 {'Extracted path': '(Other protozoal intestinal diseases)<-[:UMLS_TO_ICD]-(C0009421)-[:UMLS_TO_HPO_PHENOTYPE]->(Coma)'},
 {'Extracted path': 

## Ontology Mapping with LLMs

### Candidate Selection

In [16]:
user_query = "Shortness of breath"
q_embed = api.post('/embed', {'input': [user_query]})[0]['data'][0]
k = 10

result = graph_client.query(
    """
    CALL db.index.vector.queryNodes('hpo_phenotype_embedding', $k, $qe)
        YIELD node, score
        RETURN node.id AS id, node.label AS label, score
        ORDER BY score DESC
        LIMIT $k
    """,
    {"k": k, "qe": q_embed},
)
for rec in result:
    print(f"{rec['score']:.3f}  {rec['label']}  (id={rec['id']})")

0.897  Dyspnea  (id=HP:0002094)
0.865  Respiratory distress  (id=HP:0002098)
0.852  Tachypnea  (id=HP:0002789)
0.822  Breathing dysregulation  (id=HP:0005957)
0.821  Exertional dyspnea  (id=HP:0002875)
0.817  Respiratory insufficiency  (id=HP:0002093)
0.813  Rest dyspnea  (id=HP:0033710)
0.809  Paroxysmal dyspnea  (id=HP:0012763)
0.802  Wheezing  (id=HP:0030828)
0.802  Abnormal pattern of respiration  (id=HP:0002793)


### Candidate Disambiguation

In [39]:
payload = {
    "source_concept": "R06.0 Dyspnoea",
    "source_context": """
    {
      "id": "R06.0",
      "name": "Dyspnoea",
      "parentName": "Abnormalities of breathing",
      "group":   { "groupName": "Symptoms and signs involving the circulatory and respiratory systems" },
      "chapter": { "chapterName": "Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified" }
    }
    """,
    "candidate_list": """
    [
      {
        "score": 0.897,
        "id": "HP:0002094",
        "label": "Dyspnea",
        "exactSynonym": [
          "Shortness of breath",
          "Breathlessness"
        ],
        "description": "A subjective sensation of difficult or uncomfortable breathing."
      },
      {
        "score": 0.865,
        "id": "HP:0002098",
        "label": "Respiratory distress",
        "exactSynonym": [
          "Distressed breathing"
        ],
        "description": "A clinical state characterized by labored or difficult breathing that may be accompanied by use of accessory muscles."
      },
      {
        "score": 0.852,
        "id": "HP:0002789",
        "label": "Tachypnea",
        "exactSynonym": [
          "Increased respiratory rate",
          "Rapid breathing"
        ],
        "description": "An abnormally high respiratory rate for age at rest."
      },
      {
        "score": 0.822,
        "id": "HP:0005957",
        "label": "Breathing dysregulation",
        "exactSynonym": [
          "Disordered breathing regulation"
        ],
        "description": "Abnormal control or patterning of the respiratory rhythm."
      },
    ]
    """,
}


tool_output = build_ontology_mapper_tool(chat_client).invoke(payload)
show_json(tool_output)

```json
{
  "best_id": "HP:0002094",
  "best_label": "Dyspnea",
  "confidence": 0.9,
  "rationale": "The source concept 'Dyspnoea' is a direct synonym of 'Dyspnea' in the HP ontology.",
  "support": {
    "evidence": "Direct synonym in HP ontology.",
    "reason": "The source concept and the candidate concept are semantically equivalent."
  }
}
```

In [45]:
payload = {
    "source_concept": "R07.1 Chest pain on breathing",
    "source_context": """
    {
      "id": "R07.1",
      "name": "Chest pain on breathing",
      "parentName": "Pain in throat and chest",
      "group":   { "groupName": "Symptoms and signs involving the circulatory and respiratory systems" },
      "chapter": { "chapterName": "Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified" }
    }
    """,
    "candidate_list": """
    [
      {
        "score": 0.93,
        "id": "HP:0002104",
        "label": "Chest pain",
        "exactSynonym": [
          "Thoracic pain"
        ],
        "description": "Pain localized to the anterior or posterior chest wall, without specification of the precipitating factor, temporal pattern, or relationship to respiration or exertion. This term is intentionally broad and may encompass musculoskeletal, cardiac, pulmonary, gastrointestinal, or idiopathic etiologies when the clinical context is not further specified."
      },
      {
        "score": 0.90,
        "id": "HP:0100749",
        "label": "Exertional chest pain",
        "exactSynonym": [
          "Chest pain on exertion"
        ],
        "description": "Chest discomfort or pain that is primarily triggered, precipitated, or worsened by physical activity or emotional stress and tends to improve with rest. This feature is classically associated with myocardial ischemia or other cardiopulmonary limitations related to increased workload, rather than with respiratory movements such as inspiration or coughing."
      },
      {
        "score": 0.87,
        "id": "HP:0030165",
        "label": "Pleuritic chest pain",
        "exactSynonym": [
          "Pleural pain"
        ],
        "description": "A sharp, stabbing, or burning chest pain that is characteristically exacerbated by deep inspiration, coughing, sneezing, or other movements of the chest wall and diaphragm, and often relieved by lying on the affected side. This symptom is typically associated with inflammation or irritation of the pleura (e.g., pleuritis, pulmonary embolism, or pneumonia) and corresponds clinically to chest pain that occurs specifically on breathing."
      }
    ]
    """,
}


tool_output = build_ontology_mapper_tool(chat_client).invoke(payload)
show_json(tool_output)

```json
{
  "best_id": "HP:0030165",
  "best_label": "Pleuritic chest pain",
  "confidence": 0.93,
  "rationale": "The source concept describes chest pain on breathing, which is a key feature of pleuritic chest pain. The description of pleuritic chest pain aligns with the source concept's clinical context.",
  "support": {
    "evidence": "The description of pleuritic chest pain includes 'chest pain on breathing'.",
    "reason": "Pleuritic chest pain is characterized by sharp, stabbing pain exacerbated by breathing, which directly corresponds to the source concept's description."
  }
}
```

### Ontology Mapping Results

In [19]:
result =graph_client.query("""
    MATCH (d:IcdDisease)-[r:ICD_MAPS_TO_HPO_PHENOTYPE]->(p:HpoPhenotype)
    RETURN d.id as icd_id,
           d.label as icd_label,
           p.id AS hpo_id,
           p.label AS hpo_label,
           r.confidence AS confidence,
           r.evidence AS evidence,
           r.rationale AS rationale
    OFFSET 100
    LIMIT 1
""")
for rec in result:
    print(f"(ICD ID: {rec['icd_id']} ({rec['icd_label']})\n(HPO ID: {rec['hpo_id']}) ({rec['hpo_label']})\n(evidence={rec['evidence']}\nconfidence={rec['confidence']}\nrationale={rec['rationale']})")

(ICD ID: A18.3 (Tuberculosis of intestines, peritoneum and mesenteric glands)
(HPO ID: HP:0034841) (Gastrointestinal granulomatosis)
(evidence=None
confidence=0.8
rationale=The source concept describes tuberculosis of the intestines, peritoneum, and mesenteric glands. Gastrointestinal granulomatosis is a known manifestation of tuberculosis involving the gastrointestinal tract, which aligns with the source concept.)


In [20]:
graph_client.query("""
    MATCH path = (d:IcdDisease)<-[:UMLS_TO_ICD]-(:UMLS)-[:UMLS_TO_HPO_PHENOTYPE]->(p:HpoPhenotype)
    WHERE d.id = 'A18.3' and p.id = 'HP:0034841'
    RETURN path
""")

[]

# Patient Annotation

### Named Entity Recognition

DO NOT TRUST THE CHARACTER INDEX!

In [46]:
payload = {
    "patient_id": "patient_002",
    "encounter_id": "enc_145",
    "icd_chapters": [
        "Certain infectious and parasitic diseases",
        "Neoplasms",
        "Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism",
        "Endocrine, nutritional and metabolic diseases",
        "Mental and behavioural disorders",
        "Diseases of the nervous system",
        "Diseases of the eye and adnexa",
        "Diseases of the ear and mastoid process",
        "Diseases of the circulatory system",
        "Diseases of the respiratory system",
        "Diseases of the digestive system",
        "Diseases of the skin and subcutaneous tissue",
        "Diseases of the musculoskeletal system and connective tissue",
        "Diseases of the genitourinary system",
        "Pregnancy, childbirth and the puerperium",
        "Certain conditions originating in the perinatal period",
        "Congenital malformations, deformations and chromosomal abnormalities",
        "Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified",
        "Injury, poisoning and certain other consequences of external causes",
        "External causes of morbidity and mortality",
        "Factors influencing health status and contact with health services",
        "Codes for special purposes"
    ],
    "concat_text": (
        "Abdominal pain | "
        "Shortness of breath and chest tightness | "
        "Episodic dizziness and palpitations | "
        "Hypertension, type 2 diabetes mellitus, and iron-deficiency anemia | "
        "No Chest pain"
    ),
    "narrative_text": (
        "No narrative text provided."
    )
}


tool_output = build_patient_ner_tool(chat_client).invoke(payload)
show_json(tool_output)

```json
{
  "patient_id": "patient_002",
  "encounter_id": "enc_145",
  "entities": [
    {
      "source": "concat",
      "start": 14,
      "end": 26,
      "text": "Abdominal pain",
      "label": "Diseases of the digestive system",
      "assertion": "present",
      "temporality": "unspecified",
      "rationale": "Present in the concatenated text."
    },
    {
      "source": "concat",
      "start": 30,
      "end": 42,
      "text": "Shortness of breath and chest tightness",
      "label": "Diseases of the respiratory system",
      "assertion": "present",
      "temporality": "unspecified",
      "rationale": "Present in the concatenated text."
    },
    {
      "source": "concat",
      "start": 46,
      "end": 61,
      "text": "Episodic dizziness and palpitations",
      "label": "Diseases of the nervous system",
      "assertion": "present",
      "temporality": "unspecified",
      "rationale": "Present in the concatenated text."
    },
    {
      "source": "concat",
      "start": 65,
      "end": 88,
      "text": "Hypertension, type 2 diabetes mellitus, and iron-deficiency anemia",
      "label": "Endocrine, nutritional and metabolic diseases",
      "assertion": "present",
      "temporality": "unspecified",
      "rationale": "Present in the concatenated text."
    },
    {
      "source": "concat",
      "start": 92,
      "end": 97,
      "text": "No Chest pain",
      "label": "Diseases of the circulatory system",
      "assertion": "negated",
      "temporality": "unspecified",
      "rationale": "The patient denies chest pain."
    },
    {
      "source": "narrative",
      "start": 0,
      "end": 13,
      "text": "No narrative text provided.",
      "label": "Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified",
      "assertion": "unspecified",
      "temporality": "unspecified",
      "rationale": "No narrative text provided. This is a placeholder for a potential symptom or sign that may be present but not explicitly mentioned in the narrative."
    }
  ]
}
```

### Named Entity Disambiguation

In [47]:
ned_payload = {
    "mention": {
        "source": "concat",
        "start": 245,
        "end": 260,
        "text": "terminal ileitis",
        "label": "Diseases of the digestive system",
        "assertion": "present",
        "temporality": "chronic",
        "rationale": (
            "Imaging and colonoscopy describe inflammation of the terminal ileum. "
            "Clinical context (chronic diarrhea, weight loss) "
            "suggests inflammatory bowel disease rather than acute infection."
        ),
    },
    "candidates": [
        {
            "score": 0.912,
            "id": "K52.9",
            "label": "Noninfective gastroenteritis and colitis, unspecified"
        },
        {
            "score": 0.887,
            "id": "A09.0",
            "label": "Infectious gastroenteritis and colitis, unspecified"
        },
        {
            "score": 0.871,
            "id": "K52.0",
            "label": "Gastroenteritis and colitis due to radiation"
        },
        {
            "score": 0.842,
            "id": "K50.00",
            "label": "Crohn's disease of small intestine without complications"
        },
        {
            "score": 0.824,
            "id": "K50.80",
            "label": "Crohn's disease of both small and large intestine without complications"
        },
        {
            "score": 0.801,
            "id": "K51.90",
            "label": "Ulcerative colitis, unspecified, without complications"
        },
    ],
    "other_mentions": [
        {
            "text": "long-standing Crohn disease diagnosed at age 19",
            "label": "Diseases of the digestive system",
        },
        {
            "text": "skip lesions in terminal ileum and ascending colon on colonoscopy",
            "label": "Diseases of the digestive system",
        },
        {
            "text": "chronic watery diarrhea and 7 kg unintentional weight loss in 6 months",
            "label": "Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified",
        },
        {
            "text": "non-caseating granulomas on ileal biopsy",
            "label": "Diseases of the digestive system",
        },
    ],
}

# Invoke the Patient NED tool
ned_tool_output = build_patient_ned_tool(chat_client).invoke(ned_payload)
show_json(ned_tool_output)


```json
{
  "source": "concat",
  "start": 245,
  "end": 260,
  "text": "terminal ileitis",
  "label": "Diseases of the digestive system",
  "assertion": "present",
  "temporality": "chronic",
  "rationale": "The mention describes inflammation of the terminal ileum, which is consistent with terminal ileitis. The chronic nature of the condition is also supported by the other mentions.",
  "icd_id": "K50.00",
  "icd_label": "Crohn's disease of small intestine without complications",
  "confidence": 0.95,
  "linking_rationale": "The mention describes inflammation of the terminal ileum, which is consistent with terminal ileitis. The chronic nature of the condition is also supported by the other mentions."
}
```

## Question and Answers

### General Medical Questions (Ontology-Driven)

In [ ]:
questions = [
    #"For cystic fibrosis, what symptoms are documented and what is the evidence supporting them?",
    #"Which phenotypes of Marfan syndrome have published clinical study evidence, and what are their frequencies?",
    #"For Duchenne muscular dystrophy, list all associated symptoms and include the supporting publications.",
    "Which diseases are associated with short stature? Show me the reported frequencies.",
    #"List all phenotypes of Rett syndrome with their frequencies and PubMed URLs.",
    # "Which diseases are linked to the phenotype 'muscle weakness' supported by published clinical studies?",
    #"Find diseases associated with 'recurrent infections' and show the evidence descriptions.",
    #"Which diseases have phenotypes with adult onset and what sources support them?", # Useful to show different use cases
    #"Which diseases are treated with insulin?",
    "Find phenotypes associated with the avian influenza disease",
    # "For the ICD disease 'Typhoid fever', what HPO phenotypes are mapped and with what confidence?",
    "What is the average confidence of all HPO mappings to ICD for Cholera?",
    "List diseases associated with recurrent fever supported by published clinical studies only.",
]

for q in questions:
    print("\n=== Question ===")
    print(q)
    general_medical_tool_output = build_general_medical_tool(chat_client, debug=True).invoke({
        "question": q,
        "top_k": 10
    })
    print("=== Output ===")
    print(general_medical_tool_output["explanation"])



=== Question ===
Which diseases are associated with short stature? Show me the reported frequencies.
=== text2cypher_pipeline: START ===
Question: 'Which diseases are associated with short stature? Show me the reported frequencies.'
Step 1: Generated Cypher:
MATCH (p:HpoPhenotype)
WHERE toLower(p.label) CONTAINS toLower("short stature")
       OR toLower(p.hasExactSynonym) CONTAINS toLower("short stature")
       OR toLower(p.hasNarrowSynonym) CONTAINS toLower("short stature")
MATCH (d:HpoDisease)-[r:HAS_PHENOTYPIC_FEATURE]->(p)
RETURN
     d.id AS disease_id,
     d.label AS disease_label,
     p.id AS hpo_id,
     p.label AS hpo_label,
     p.comment AS hpo_comment,
     r.frequency AS frequency,
     r.source AS source,
     r.url AS pubmed_url
ORDER BY d.label
Step 2: EXPLAIN on generated query: OK
Step 3: Relationship correction: no change
Step 4: LLM validation: no errors found
Step 5: No correction needed
Step 6: Executing final Cypher query:
MATCH (p:HpoPhenotype)
WHERE toLowe

### Patient Tool (Virtualized Resources)

In [24]:
patient_payload = {
    "patient_id": "P003",
    #"question": "Provide a concise clinical summary of patient P003 using all documented findings.",
    "question": "Provide me details on the follow up plan of patient P003 in the latest encounter.",
}

patient_tool_output = build_patient_info_tool(chat_client).invoke(patient_payload)
patient_tool_output

{'patient_id': 'P003',
 'question': 'Provide me details on the follow up plan of patient P003 in the latest encounter.',
 'encounter_date': 'latest',
 'has_data': True,
 'answer': 'Answer:\nThe follow-up plan for patient P003 in the latest encounter on 2026-03-03 is to monitor for new skin changes and reassess if the lesions spread. The patient is instructed to use regular emollients.\n',
 'raw_patient_view': [{'patient_id': 'P003',
   'identity': None,
   'labels': [],
   'elementId': None,
   'condition': 'Striae distensae',
   'chief_complaint': 'Thin streak-like marks on the trunk',
   'course_trend': 'Stable',
   'comorbidities': 'Multiple mononeuropathy, recurrent inflammatory episodes',
   'plan_followup': 'Monitor for new skin changes and reassess if lesions spread',
   'medication_statement': 'Emollient cream topically BID',
   'notes': 'Skin findings unchanged since onset',
   'encounter': {'id': 'E012',
    'period_start': '2026-03-03',
    'reason_code': 'New skin streaks o

### Coverage Tool (using the ICD-HPO mappings)

In [25]:
graph_client.query("""
    CALL apoc.dv.query('patient', {patientId: 'P003'}) YIELD node AS v
    WITH apoc.convert.fromJsonList(
           coalesce(
               apoc.any.property(v, 'ICD10_Codes'),
               apoc.any.property(v, '\uFEFFICD10_Codes')
           )
         ) AS codes
    RETURN codes
    LIMIT 1
""")

[{'codes': ['G57.1',
   'T28.1',
   'S84',
   'R52',
   'G58.7',
   'S94.7',
   'R26.2',
   'M54.8',
   'R26.0',
   'G58.7']}]

In [26]:
graph_client.query("""
    CALL apoc.dv.query('patient', {patientId: 'P003'}) 
    YIELD node AS v

    WITH
    apoc.convert.fromJsonList(
        coalesce(
        apoc.any.property(v, 'ICD10_Codes'),
        apoc.any.property(v, '\uFEFFICD10_Codes')
        )
    ) AS codes,
    v AS _Patient
    WITH codes, _Patient
    LIMIT 1

    UNWIND codes AS code

    MATCH (d:IcdDisease {id: code})
        <-[:UMLS_TO_ICD]-
        (u:UMLS)
        -[:UMLS_TO_HPO_PHENOTYPE]->
        (h:HpoPhenotype)

    RETURN DISTINCT
    apoc.any.property(_Patient, 'PatientId') AS PatientId,
    h.label AS HPO_Label,
    h.id AS hpo_id,
    // Fancy ASCII path description
    'Patient[' + apoc.any.property(_Patient, 'PatientId') + ']'
    + ' --ICD10[' + code + ']--> '
    + 'HPO[' + h.id + ' | ' + h.label + ']' AS PathDescription;

""")

[{'PatientId': 'P003',
  'HPO_Label': 'Pain',
  'hpo_id': 'HP:0012531',
  'PathDescription': 'Patient[P003] --ICD10[R52]--> HPO[HP:0012531 | Pain]'},
 {'PatientId': 'P003',
  'HPO_Label': 'Multiple mononeuropathy',
  'hpo_id': 'HP:0032018',
  'PathDescription': 'Patient[P003] --ICD10[G58.7]--> HPO[HP:0032018 | Multiple mononeuropathy]'},
 {'PatientId': 'P003',
  'HPO_Label': 'Gait ataxia',
  'hpo_id': 'HP:0002066',
  'PathDescription': 'Patient[P003] --ICD10[R26.0]--> HPO[HP:0002066 | Gait ataxia]'}]

In [27]:
coverage_payload = {
    "patient_id": "P003",
    "limit": 20
}

coverage_tool_output = build_patient_coverage_tool(chat_client).invoke(coverage_payload)
coverage_tool_output


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 2, column: 5, offset: 5} for query: '\n    CALL {\n        UNWIND $codes AS code\n        MATCH (:IcdDisease {id: code})-[:ICD_MAPS_TO_HPO_PHENOTYPE]->(h:HpoPhenotype)\n        RETURN DISTINCT h.id AS hpo_id\n        \n        UNION\n        \n        UNWIND $codes AS code\n        MATCH (:IcdDisease {id: code})<-[:UMLS_TO_ICD]-(:UMLS)-[:UMLS_TO_HPO_PHENOTYPE]->(h:HpoPhenotype)\n        RETURN DISTINCT h.id AS hpo_id\n    }\n    RETURN collect(DISTINCT hpo_id) AS hpo_ids;\n    '


{'cypher': 'MULTI-STEP PIPELINE: get_patient_icd_codes -> map_icd_to_hpo -> rollup_hpo_to_ancestors -> compute_coverage',
 'rows': [{'diseaseId': 'ORPHA:117',
   'diseaseName': 'Behçet disease',
   'covered': 16,
   'total': 85,
   'coveragePct': 18.8,
   'missingHpoIds': ['HP:0002204',
    'HP:0002202',
    'HP:0007813',
    'HP:0200039',
    'HP:0001637',
    'HP:0100820',
    'HP:0002376',
    'HP:0001482',
    'HP:0000031',
    'HP:0000155',
    'HP:0002014',
    'HP:0001289',
    'HP:0000518',
    'HP:0100584',
    'HP:0100614',
    'HP:0012378',
    'HP:0100653',
    'HP:0200034',
    'HP:0100654',
    'HP:0000083',
    'HP:0002383',
    'HP:0001250',
    'HP:0012219',
    'HP:0002024',
    'HP:0001097',
    'HP:0012819',
    'HP:0002027',
    'HP:0001653',
    'HP:0000488',
    'HP:0002105',
    'HP:0002102',
    'HP:0001658',
    'HP:0001733',
    'HP:0001659',
    'HP:0100326',
    'HP:0010885',
    'HP:4000041',
    'HP:0001061',
    'HP:0000099',
    'HP:0002076',
    'HP:00

# GraphRAG Agent

In [28]:
from llm.agent import run_agent


In [29]:
out = run_agent("Find phenotypes associated with the influenza disease and include the mapping support text.")
print(out["final_answer"])

The following phenotypes are associated with influenza disease:

*   **Acute bronchitis:** A common type of bronchitis, often caused by Haemophilus influenzae.
*   **Acute infectious pneumonia:** A common manifestation of influenza, especially in the lungs.
*   **Bacteremia:** The presence of bacteria in the bloodstream, which can occur with Haemophilus influenzae infections.
*   **Bronchitis:** A type of inflammation of the bronchial tubes, often associated with influenza.
*   **Pneumonia:** Lung infection, which can be caused by influenza.
*   **Severe influenza infection:** A more severe form of influenza.
*   **Triggered by vaccination:** Can be a side effect of the influenza vaccine.
*   **Unusual viral infection:** Influenza can be considered an unusual viral infection if the virus is not identified.


In [30]:
out = run_agent("Provide me details on the follow up plan of patient P003 in the latest encounter.")
print(out["final_answer"])

Answer:
The follow-up plan for patient P003 in the latest encounter on 2026-03-03 is to monitor for new skin changes and reassess if the lesions spread. The patient is instructed to use regular emollients.



In [31]:
out = run_agent("Show possible diseases by HPO coverage for patientId:'P003'. Report the covered, total, and percentage coverage.")
print(out["final_answer"])

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 2, column: 5, offset: 5} for query: '\n    CALL {\n        UNWIND $codes AS code\n        MATCH (:IcdDisease {id: code})-[:ICD_MAPS_TO_HPO_PHENOTYPE]->(h:HpoPhenotype)\n        RETURN DISTINCT h.id AS hpo_id\n        \n        UNION\n        \n        UNWIND $codes AS code\n        MATCH (:IcdDisease {id: code})<-[:UMLS_TO_ICD]-(:UMLS)-[:UMLS_TO_HPO_PHENOTYPE]->(h:HpoPhenotype)\n        RETURN DISTINCT h.id AS hpo_id\n    }\n    RETURN collect(DISTINCT hpo_id) AS hpo_ids;\n    '


DEBUG: Retrieved ICD codes: ['A68', 'A68.9', 'B48.8', 'F40.0', 'F43.8', 'F51.4', 'F59', 'F90.0', 'G02.1', 'G03.0', 'G44', 'G44.2', 'G57.1', 'G58.7', 'H30.0', 'H30.2', 'H35.0', 'H43.0', 'H49.1', 'H49.2', 'H49.3', 'H50.2', 'H53.2', 'H53.9', 'H57.1', 'I44.0', 'I47', 'I48.4', 'I49.4', 'K03.0', 'K07.2', 'K07.6', 'L90.6', 'M13.1', 'M23.4', 'M24.5', 'M25.4', 'M25.5', 'M25.6', 'M54.2', 'M54.8', 'R00.2', 'R11', 'R22.4', 'R23.8', 'R25.0', 'R26.0', 'R26.2', 'R41.1', 'R41.8', 'R42', 'R50', 'R50.8', 'R51', 'R52', 'R52.9', 'R53', 'R55', 'R70.0', 'R83.4', 'S03.4', 'S13.4', 'S84', 'S94.7', 'T03.0', 'T28.1', 'W10', 'Z50']
Based on the provided data, the following diseases are covered for patientId 'P003': Behçet disease (16/85, 18.8%), Lyme disease (15/24, 62.5%), Brucellosis (15/77, 19.5%), Developmental delay, impaired speech, and behavioral abnormalities (15/187, 8.0%), and Microscopic polyangiitis (11/55, 20.0%).



# Embedding API

In [32]:
cfg = load_config_api("embedding", path="../config.ini")
api = ApiClient(cfg)

In [33]:
body, status, headers = api.post('/embed', {'input': ['Who are you?', 'What is your name?']})
print("Status:", status)
print("Headers:", headers)
print("Body:", body)

Status: 200
Headers: {'Content-Length': '153883', 'Content-Type': 'application/json', 'Date': 'Wed, 19 Nov 2025 13:28:56 GMT', 'Server': 'uvicorn', 'Connection': 'close'}
Body: {'model': 'Alibaba-NLP/gte-Qwen2-7B-instruct', 'num_inputs': 2, 'embedding_dims': 3584, 'data': [[0.006518279667943716, 0.0009311828180216253, -0.00012745565618388355, 0.01802769862115383, 0.003128774231299758, -0.009758795611560345, 0.013558021746575832, 0.0015736989444121718, -0.021901419386267662, 0.02413625828921795, 0.007561204489320517, -0.014973419718444347, 0.004916645120829344, -0.006406537722796202, 0.023242322728037834, 0.010503741912543774, 0.02160344086587429, -0.019666580483317375, -0.007002494763582945, -0.00506563438102603, 0.019666580483317375, 0.0021975913550704718, -0.0076356991194188595, -0.00030496236286126077, 0.006294795777648687, -0.016090840101242065, -0.015420387499034405, -0.0324796587228775, 0.004432430025190115, 0.02309333346784115, 0.01720825769007206, 0.004283440764993429, 0.002057

# Semantic Search

In [34]:
import configparser
from database.neo4j_db import Neo4jGraphDB

config = configparser.ConfigParser()
config.read('../config.ini')
neo4j_graph = Neo4jGraphDB(database=config["neo4j"]["database"])
driver = neo4j_graph._driver
with driver.session(database=neo4j_graph._database) as session:
    result = session.run("MATCH (n) RETURN count(n) AS node_count")
    node_count = result.single()["node_count"]
print(f"Total number of nodes in the Neo4j database: {node_count}")


Total number of nodes in the Neo4j database: 93325


In [35]:
user_query = "Muscular dystrophy."
q_embed = api.post('/embed', {'input': [user_query]})[0]['data'][0]
k = 10

with driver.session(database=neo4j_graph._database) as sess:
    result = sess.run("""
        CALL db.index.vector.queryNodes('hpo_phenotype_embedding', $k, $qe)
        YIELD node, score
        RETURN node.id as id, node.label as label, score
        ORDER BY score DESC
        LIMIT $k
    """, k=k, qe=q_embed)
    for rec in result:
        print(f"{rec['score']:.3f}  {rec['label']}  (id={rec['id']})")

0.875  Muscular dystrophy  (id=HP:0003560)
0.811  Skeletal myopathy  (id=HP:0003756)
0.806  Progressive spinal muscular atrophy  (id=HP:0009067)
0.801  Myositis  (id=HP:0100614)
0.796  Limb-girdle muscle atrophy  (id=HP:0003797)
0.792  Amyoplasia  (id=HP:0003634)
0.785  Myopathy  (id=HP:0003198)
0.783  Necrotizing myopathy  (id=HP:0008978)
0.783  Myofibrillar myopathy  (id=HP:0003715)
0.779  Amyotrophic lateral sclerosis  (id=HP:0007354)
